# Random forest 

1. Load the merged_df dataset. 

2. Split it into train and test: This step is crucial for evaluating the model's performance. We'll use the 70-30 split, and we'll use X_train_res and y_train_res for training and X_test and y_test for testing.

3. Apply SMOTE on the train set: We'll apply SMOTE only to the training set (X_train, y_train) to address class imbalance, resulting in X_train_res and y_train_res.

4. Create the pipeline with RFE and the model: We'll create a pipeline that includes Recursive Feature Elimination (RFE) and the chosen model. This pipeline will be trained on the resampled training set (X_train_res, y_train_res).

5. Hyperparameter tuning using grid search: We'll perform hyperparameter tuning using grid search within an outer loop of cross-validation. Since grid search involves training multiple models with different hyperparameter combinations, it should be performed on the resampled training set (X_train_res, y_train_res).

6. Metrics to evaluate: Finally, we'll evaluate the model's performance using various metrics on the test set (X_test, y_test). Common metrics include accuracy, precision, recall, F1-score, and ROC AUC score.


Import libraries. 

In [1]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

Complete_dataset is the dataframe I created with all the extracted features from TS fresh. 

In [4]:
complete_dataset = pd.read_csv('/Users/dionnespaltman/Desktop/V4/complete_dataset.csv', sep=',')
complete_dataset.drop('Unnamed: 0', axis=1, inplace=True)
display(complete_dataset.head(5))

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU09_r__sum_values,AU09_r__variance,AU09_r__standard_deviation,AU09_r__maximum,AU09_r__minimum,AU09_r__median,AU09_r__mean,AU09_r__mean_abs_change,"AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min"""
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,1772.86,0.109496,0.330902,4.98,0.00,0.00,0.120554,0.019213,-0.000032,-0.000022
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,3197.59,0.063365,0.251725,2.59,0.00,0.00,0.117537,0.024182,-0.000029,-0.000019
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,2275.65,0.153114,0.391297,4.10,0.00,0.00,0.138912,0.024858,0.000048,0.000033
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,1755.79,0.052783,0.229745,3.25,0.00,0.00,0.099096,0.016996,-0.000013,-0.000012
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,20877.08,0.779602,0.882951,3.45,-4.43,1.05,0.992115,0.150508,0.000089,0.000095


I want to know how many people are in each group. 

In [5]:
# Count the number of instances of people in VVR_group = 1 and VVR_group = 0
count_vvr_group = complete_dataset['VVR_group'].value_counts()

# Print the counts
print("Number of instances in VVR_group = 1:", count_vvr_group[1])
print("Number of instances in VVR_group = 0:", count_vvr_group[0])

Number of instances in VVR_group = 1: 23
Number of instances in VVR_group = 0: 81


In [6]:
columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition'] 

In [5]:
# X = merged_df.drop(columns_to_drop, axis=1)
# y = merged_df['VVR_group']

These are the columns I use to predict, so all my features. I need these as a list to establish my featurizer. 
I have 119 features from TS fresh and then I added the two VVR measurements from stage 1 and 2. 

In [5]:
with open('/Users/dionnespaltman/Desktop/V3/columns_au_12.json', 'r') as f:
    columns_au_12 = json.load(f)

print(len(columns_au_12))

121


First we'll split the data into a train and test set. 

In total, we have 111 participants.

I started with a test size of 20%. Then there are only 23 people in the test set (0: 18, 1: 5). 
Then I made the test size 30%. Then there are 34 people in the test set (0: 26, 1: 8). 
The best results are however if the test_size is 40%. Then we have 45 participants in the test set (0: 34, 1: 11). 

Naturally, we stratify on VVR_group. 

In [11]:
train, test = train_test_split(merged_df, test_size=0.3, random_state=123, stratify=merged_df['VVR_group'])

print(train.shape)
print(test.shape)

(77, 127)
(34, 127)


Unfortunately, the test set is very small with only 8 people in the high VVR condition. 

In [12]:
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

X_test = test.drop(columns_to_drop, axis=1)
y_test = test['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_test))

Original dataset shape Counter({0: 26, 1: 8})


# Applying SMOTE

In [13]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

We apply SMOTE on the train data. The strategy is to make the minority as big as the majority class. 

In [14]:
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

X_train = train.drop(columns_to_drop, axis=1)
y_train = train['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_train))

# Apply SMOTE to the training data with sampling strategy set to 'auto' (default)
sm = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Print resampled class distribution
print('Resampled dataset shape %s' % Counter(y_train_res))

Original dataset shape Counter({0: 59, 1: 18})
Resampled dataset shape Counter({1: 59, 0: 59})


# Adding class weights

I will add class weights to my models, because of my inbalanced data set. 
https://medium.com/@ravi.abhinav4/improving-class-imbalance-with-class-weights-in-machine-learning-af072fdd4aa4 

Results: 
Class weights: {0: 0.6470588235294118, 1: 2.2}

In [15]:
import numpy as np

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    class_weights = {}

    for class_label, class_count in zip(unique_classes, class_counts):
        class_weight = total_samples / (2.0 * class_count)
        class_weights[class_label] = class_weight

    return class_weights

# Assuming 'y' contains the class labels (0s and 1s) for the binary classification problem
class_weights = calculate_class_weights(y_train)
print("Class weights:", class_weights)

# this is how you would implement it 
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], class_weight=class_weights)

Class weights: {0: 0.652542372881356, 1: 2.138888888888889}


# Featurizer 

Here I make the featurizer. 

In [16]:
featurizer = ColumnTransformer(transformers=[("numeric", StandardScaler(), columns_au_12)], remainder='drop')

# Model (not complete but a intermediate step)

Strange enough the recall is much higher (0.27 instead of 0.18) when I don't use class weights. 

The model you can see here is not using K-fold cross validation. 

Here I haven't done any Recursive Feature Analysis or cross validation. 

In [14]:
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

# model = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0, class_weight=class_weights))
model = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0))

# here we fit the model on the resampled X train (so the balanced dataset due to SMOTE)
model.fit(X_train_res, y_train_res)

print('Original dataset shape %s' % Counter(y_train_res))

# Then we predict 
pred = model.predict(test.drop('VVR_group', axis=1))

# The metrics 
accuracy = accuracy_score(test['VVR_group'].values, pred)
report = classification_report(test['VVR_group'].values, pred)
cm = confusion_matrix(test['VVR_group'].values, pred)
precision, recall, _ = precision_recall_curve(test['VVR_group'].values, pred)
auc_pr = auc(recall, precision)

# Print results
print(f"Accuracy on Validation Data: {accuracy}")
print(f"AUC-PR on Validation Data: {auc_pr}")
print("Classification Report:")
print(report)

Original dataset shape Counter({1: 51, 0: 51})
Accuracy on Validation Data: 0.7777777777777778
AUC-PR on Validation Data: 0.5252525252525253
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.94      0.86        34
           1       0.60      0.27      0.38        11

    accuracy                           0.78        45
   macro avg       0.70      0.61      0.62        45
weighted avg       0.75      0.78      0.75        45



# Recursive feature elimination to optimize the scores - not using SMOTE

I based my code partially on the following article: 
https://machinelearningmastery.com/rfe-feature-selection-in-python/

Here I did apply cross validation, but not yet with hyperparameter tuning, so it's not the final model yet. 

For this step we don't use SMOTE because you can't specify what the test and train set is. So here I make a X and y and use this. 

In [17]:
merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V3/merged_df.csv', sep=',')

merged_df.drop('Unnamed: 0', axis=1, inplace=True)
merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)

In [18]:
# columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition', 'VVR_1', 'VVR_2'] 
columns_to_drop = [ 'ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition'] 

X = merged_df.drop(columns_to_drop, axis=1)
y = merged_df['VVR_group']

print(X.shape)

(111, 121)


Recall. 

In [19]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='recall', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('Recall: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Recall: 0.500 (0.295)


Precision. 

In [20]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='precision', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('Precision: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Precision: 0.436 (0.269)


/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy. 

In [21]:
# evaluate RFE for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.722 (0.106)


# Model including RFE and hyperparameter tuning in Grid Search (not yet with inner and outer split!)

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Create the pipeline with RFE and the model
rfe_new = RFE(estimator=RandomForestClassifier(), n_features_to_select=10)
model_new = RandomForestClassifier()
pipeline_new = Pipeline(steps=[('s', rfe_new), ('m', model_new)])

# Hyperparameter tuning using grid search
param_grid = {
    'm__n_estimators': [100, 200, 300],  # Number of trees in the forest
    # 'm__max_depth': [None, 10, 20],  # Maximum depth of the tree
    # 'm__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    # 'm__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    # 's__n_features_to_select': [5, 10, 15]  # Number of features to select with RFE
}
grid_search = GridSearchCV(pipeline_new, param_grid, cv=5, scoring='recall', n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

from sklearn.model_selection import cross_val_score, StratifiedKFold

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X_train_res, y_train_res, cv=cv, scoring='recall')

# Print cross-validation scores
print("Cross-validation scores: ", cv_scores)
print("Mean CV recall: ", cv_scores.mean())
print("Standard deviation of CV recall: ", cv_scores.std())

# Print best parameters
print("Best parameters found:")
print(grid_search.best_params_)

# Metrics to evaluate
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

from sklearn.metrics import roc_auc_score

# Compute AUC-ROC per class
y_pred_proba = best_model.predict_proba(X_test)  # Get predicted probabilities

print(y_pred_proba.shape)
print(y_test.shape)
# auc_per_class = roc_auc_score(y_test, y_pred_proba, average=None)

# # Print AUC-ROC per class
# print("AUC-ROC per class:")
# for i, auc in enumerate(auc_per_class):
#     print(f"Class {i}: {auc}")


Cross-validation scores:  [0.75       0.83333333 0.91666667 0.91666667 0.81818182]
Mean CV recall:  0.8469696969696969
Standard deviation of CV recall:  0.06345573209506641
Best parameters found:
{'m__n_estimators': 100}
              precision    recall  f1-score   support

           0       0.83      0.77      0.80        26
           1       0.40      0.50      0.44         8

    accuracy                           0.71        34
   macro avg       0.62      0.63      0.62        34
weighted avg       0.73      0.71      0.72        34

(34, 2)
(34,)


# Model with inner and outer split

In [32]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)

# Create the pipeline with RFE and the model
rfe_new = RFE(estimator=RandomForestClassifier(), n_features_to_select=10)
model_new = RandomForestClassifier()
pipeline_new = Pipeline(steps=[('s', rfe_new), ('m', model_new)])

param_grid = {
    'm__n_estimators': [100, 200, 300],  # Number of trees in the forest
    # 'm__max_depth': [None, 10, 20],  # Maximum depth of the tree
    # 'm__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    # 'm__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    # 's__n_features_to_select': [10, 20, 30, 40]  # Number of features to select with RFE
}

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=pipeline_new, param_grid=param_grid, cv=inner_cv, n_jobs=2
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_train_res, y_train_res, cv=outer_cv, n_jobs=2)
print(
    "The mean score using nested cross-validation is: "
    f"{test_score.mean():.3f} ± {test_score.std():.3f}"
)

# # Print best parameters
# print("Best parameters found:")
# print(model.best_estimator_.named_steps['m'].get_params())  # Adjust 'm' if necessary
# # print(model.best_params_)

# Evaluate on the test set
best_model = model.best_estimator_
y_pred = best_model.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

The mean score using nested cross-validation is: 0.822 ± 0.019


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [231]:
print(X_test.shape)
print(X_train_res.shape)

(45, 121)
(102, 121)
